<a href="https://colab.research.google.com/github/MuhammadIrzam447/MultiModel/blob/master/Valid_102.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Best Model - ViT Fused Validation
# Notebook to Evaluate ViT on 100 and __% datasets residing in a single directory.

In [1]:
import torch
!pip install transformers evaluate datasets
import requests
import torch
from PIL import Image
from transformers import *
from tqdm import tqdm
from torch.utils.data import DataLoader
device = "cuda" if torch.cuda.is_available() else "cpu"


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


/usr/local/lib/python3.10/dist-packages/transformers/generation_utils.py:24: FutureWarning: Importing `GenerationMixin` from `src/transformers/generation_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import GenerationMixin` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation_tf_utils.py:24: FutureWarning: Importing `TFGenerationMixin` from `src/transformers/generation_tf_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import TFGenerationMixin` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation_flax_utils.py:24: FutureWarning: Importing `FlaxGenerationMixin` from `src/transformers/generation_flax_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import FlaxGenerationMixin` instead.
  warnings.warn(
Xformers is not installed correctly. If you want to use memory_efficient_attentio

In [2]:
import os

test_image_file_paths = []
test_image_genre_labels = []
test_enc_file_paths = []
test_enc_genre_labels = []

test_labels_file = "/content/Dataset(s)/joint-food-101/test_dataset.txt"

with open(test_labels_file, 'r') as file:
    for line in file:
        parts = line.strip().split(', ')
        filename = parts[0].strip()
        labels = parts[1].strip()

        if filename.endswith("_4.png"):
            test_enc_file_paths.append(filename)
            test_enc_genre_labels.append(labels)

        elif filename.endswith("_3.png"):
            test_image_file_paths.append(filename)
            test_image_genre_labels.append(labels)

In [3]:
len(test_image_file_paths) , len(test_enc_file_paths)

(22716, 22716)

In [4]:
unique_labels = set(test_enc_genre_labels)
unique_labels_list = list(unique_labels)

print("Unique Labels:", unique_labels_list)

Unique Labels: ['french_toast', 'strawberry_shortcake', 'apple_pie', 'foie_gras', 'shrimp_and_grits', 'escargots', 'caesar_salad', 'spring_rolls', 'baby_back_ribs', 'takoyaki', 'cannoli', 'poutine', 'red_velvet_cake', 'prime_rib', 'pad_thai', 'tacos', 'steak', 'chocolate_cake', 'bibimbap', 'club_sandwich', 'omelette', 'dumplings', 'grilled_salmon', 'bruschetta', 'hummus', 'frozen_yogurt', 'chicken_quesadilla', 'edamame', 'waffles', 'creme_brulee', 'cheese_plate', 'panna_cotta', 'lobster_roll_sandwich', 'breakfast_burrito', 'chocolate_mousse', 'caprese_salad', 'peking_duck', 'samosa', 'ravioli', 'miso_soup', 'tuna_tartare', 'chicken_wings', 'sashimi', 'greek_salad', 'pancakes', 'pizza', 'spaghetti_bolognese', 'mussels', 'macarons', 'cup_cakes', 'fish_and_chips', 'french_fries', 'crab_cakes', 'garlic_bread', 'tiramisu', 'beef_carpaccio', 'beignets', 'lobster_bisque', 'pulled_pork_sandwich', 'ice_cream', 'ramen', 'lasagna', 'cheesecake', 'bread_pudding', 'fried_calamari', 'falafel', 'beef

In [5]:
len(unique_labels_list)

101

In [6]:
import random

percentage = 0.3

label_counts = {label: 0 for label in unique_labels_list}

test_num_samples_to_include_per_label = {}
for label in unique_labels_list:
    label_count = test_enc_genre_labels.count(label)
    num_samples_to_include = int(label_count * percentage)
    test_num_samples_to_include_per_label[label] = num_samples_to_include


test_filtered_enc_file_paths = []
test_filtered_enc_genre_labels = []


test_selected_samples_per_label = {label: 0 for label in unique_labels_list}
for path, label in zip(test_enc_file_paths, test_enc_genre_labels):
    if test_selected_samples_per_label[label] < test_num_samples_to_include_per_label[label]:
        test_filtered_enc_file_paths.append(path)
        test_filtered_enc_genre_labels.append(label)
        test_selected_samples_per_label[label] += 1

In [7]:
len(test_filtered_enc_file_paths) , len(test_filtered_enc_genre_labels)

(6770, 6770)

In [8]:
# Create new lists to combine data from both lists
test_file_paths = test_image_file_paths + test_filtered_enc_file_paths
test_genre_labels = test_image_genre_labels + test_filtered_enc_genre_labels

In [9]:
from sklearn.preprocessing import LabelEncoder

label_encoder1 = LabelEncoder()
transformed_test_genre_labels = label_encoder1.fit_transform(test_genre_labels)

In [10]:
label_to_id_mapping = {label: id for label, id in zip(label_encoder1.classes_, label_encoder1.transform(label_encoder1.classes_))}
id_to_label_mapping = {id: label for id, label in enumerate(label_encoder1.classes_)}

In [11]:
# Convert int64 to int in id_to_label_mapping and label_to_id_mapping
id_to_label_mapping = {int(id): label for id, label in id_to_label_mapping.items()}
label_to_id_mapping = {label: int(id) for label, id in label_to_id_mapping.items()}

In [12]:
id_to_label_mapping

{0: 'apple_pie',
 1: 'baby_back_ribs',
 2: 'baklava',
 3: 'beef_carpaccio',
 4: 'beef_tartare',
 5: 'beet_salad',
 6: 'beignets',
 7: 'bibimbap',
 8: 'bread_pudding',
 9: 'breakfast_burrito',
 10: 'bruschetta',
 11: 'caesar_salad',
 12: 'cannoli',
 13: 'caprese_salad',
 14: 'carrot_cake',
 15: 'ceviche',
 16: 'cheese_plate',
 17: 'cheesecake',
 18: 'chicken_curry',
 19: 'chicken_quesadilla',
 20: 'chicken_wings',
 21: 'chocolate_cake',
 22: 'chocolate_mousse',
 23: 'churros',
 24: 'clam_chowder',
 25: 'club_sandwich',
 26: 'crab_cakes',
 27: 'creme_brulee',
 28: 'croque_madame',
 29: 'cup_cakes',
 30: 'deviled_eggs',
 31: 'donuts',
 32: 'dumplings',
 33: 'edamame',
 34: 'eggs_benedict',
 35: 'escargots',
 36: 'falafel',
 37: 'filet_mignon',
 38: 'fish_and_chips',
 39: 'foie_gras',
 40: 'french_fries',
 41: 'french_onion_soup',
 42: 'french_toast',
 43: 'fried_calamari',
 44: 'fried_rice',
 45: 'frozen_yogurt',
 46: 'garlic_bread',
 47: 'gnocchi',
 48: 'greek_salad',
 49: 'grilled_chees

In [13]:
from datasets import Dataset

val_data = {'image': test_file_paths, 'label': transformed_test_genre_labels}
ds_val = Dataset.from_dict(val_data)

In [14]:
import PIL.Image as pil

def transform(examples):
  inputs = image_processor([pil.open(img).convert("RGB") for img in examples["image"]], return_tensors="pt")
  inputs["labels"] = examples["label"]
  return inputs

In [15]:
val_dataset = ds_val.with_transform(transform)

In [16]:
import torch

def collate_fn(batch):
  return {
      "pixel_values": torch.stack([x["pixel_values"] for x in batch]),
      "labels": torch.tensor([x["labels"] for x in batch]),
  }

In [17]:
batch_size = 32
valid_dataset_loader = DataLoader(val_dataset, collate_fn=collate_fn, batch_size=batch_size, shuffle=False)

In [18]:
checkpoint = 9000

In [19]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, classification_report

In [20]:
while checkpoint <= 54000:
    model = ViTForImageClassification.from_pretrained(f"/content/Model/Models-Train-29/checkpoint-{checkpoint}").to(device)
    image_processor = ViTImageProcessor.from_pretrained(f"/content/Model/Models-Train-29/checkpoint-{checkpoint}")
    model = model.to(device)
    model.eval()
    print()
    print("Evaluation at checkpoint: ", checkpoint)
    print()

    predictions, labels = [], []

    for batch in valid_dataset_loader:
        pixel_values = batch["pixel_values"].to(device)
        label_ids = batch["labels"].to(device)

        outputs = model(pixel_values=pixel_values, labels=label_ids)
        logits = outputs.logits.detach().cpu()

        predictions.extend(logits.argmax(dim=-1).tolist())
        labels.extend(label_ids.tolist())

    accuracy = accuracy_score(labels, predictions)
    precision = precision_score(labels, predictions, average='weighted')
    recall = recall_score(labels, predictions, average='weighted')
    f1 = f1_score(labels, predictions, average='weighted')

    print("Accuracy: ", accuracy)
    print("precision: ", precision)
    print("f1_score: ", f1)
    print("recall", recall)
    print(classification_report(labels, predictions))
    checkpoint = checkpoint + 9000

loading configuration file /content/Model/Models-Train-29/checkpoint-9000/config.json
Model config ViTConfig {
  "_name_or_path": "google/vit-base-patch16-224",
  "architectures": [
    "ViTForImageClassification"
  ],
  "attention_probs_dropout_prob": 0.0,
  "encoder_stride": 16,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 768,
  "id2label": {
    "0": "apple_pie",
    "1": "baby_back_ribs",
    "2": "baklava",
    "3": "beef_carpaccio",
    "4": "beef_tartare",
    "5": "beet_salad",
    "6": "beignets",
    "7": "bibimbap",
    "8": "bread_pudding",
    "9": "breakfast_burrito",
    "10": "bruschetta",
    "11": "caesar_salad",
    "12": "cannoli",
    "13": "caprese_salad",
    "14": "carrot_cake",
    "15": "ceviche",
    "16": "cheese_plate",
    "17": "cheesecake",
    "18": "chicken_curry",
    "19": "chicken_quesadilla",
    "20": "chicken_wings",
    "21": "chocolate_cake",
    "22": "chocolate_mousse",
    "23": "churros",
    "24": "clam_chowder",


Evaluation at checkpoint:  9000



loading configuration file /content/Model/Models-Train-29/checkpoint-18000/config.json
Model config ViTConfig {
  "_name_or_path": "google/vit-base-patch16-224",
  "architectures": [
    "ViTForImageClassification"
  ],
  "attention_probs_dropout_prob": 0.0,
  "encoder_stride": 16,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 768,
  "id2label": {
    "0": "apple_pie",
    "1": "baby_back_ribs",
    "2": "baklava",
    "3": "beef_carpaccio",
    "4": "beef_tartare",
    "5": "beet_salad",
    "6": "beignets",
    "7": "bibimbap",
    "8": "bread_pudding",
    "9": "breakfast_burrito",
    "10": "bruschetta",
    "11": "caesar_salad",
    "12": "cannoli",
    "13": "caprese_salad",
    "14": "carrot_cake",
    "15": "ceviche",
    "16": "cheese_plate",
    "17": "cheesecake",
    "18": "chicken_curry",
    "19": "chicken_quesadilla",
    "20": "chicken_wings",
    "21": "chocolate_cake",
    "22": "chocolate_mousse",
    "23": "churros",
    "24": "clam_chowder"

Accuracy:  0.7619548260191277
precision:  0.8096031953276193
f1_score:  0.7783889005395094
recall 0.7619548260191277
              precision    recall  f1-score   support

           0       0.82      0.82      0.82       304
           1       0.92      0.76      0.83       287
           2       0.84      0.69      0.76       293
           3       0.29      0.68      0.40       288
           4       0.24      0.55      0.33       292
           5       0.75      0.79      0.77       291
           6       0.92      0.75      0.83       291
           7       0.78      0.76      0.77       292
           8       0.87      0.88      0.88       293
           9       0.79      0.73      0.76       278
          10       0.89      0.84      0.86       300
          11       0.71      0.75      0.73       295
          12       0.80      0.84      0.82       299
          13       0.83      0.78      0.81       286
          14       0.88      0.78      0.83       300
          15      

All model checkpoint weights were used when initializing ViTForImageClassification.

All the weights of ViTForImageClassification were initialized from the model checkpoint at /content/Model/Models-Train-29/checkpoint-18000.
If your task is similar to the task the model of the checkpoint was trained on, you can already use ViTForImageClassification for predictions without further training.
loading configuration file /content/Model/Models-Train-29/checkpoint-18000/preprocessor_config.json
Image processor ViTImageProcessor {
  "do_normalize": true,
  "do_rescale": true,
  "do_resize": true,
  "image_mean": [
    0.5,
    0.5,
    0.5
  ],
  "image_processor_type": "ViTImageProcessor",
  "image_std": [
    0.5,
    0.5,
    0.5
  ],
  "resample": 2,
  "rescale_factor": 0.00392156862745098,
  "size": {
    "height": 224,
    "width": 224
  }
}



...


Evaluation at checkpoint:  45000



loading configuration file /content/Model/Models-Train-29/checkpoint-54000/config.json
Model config ViTConfig {
  "_name_or_path": "google/vit-base-patch16-224",
  "architectures": [
    "ViTForImageClassification"
  ],
  "attention_probs_dropout_prob": 0.0,
  "encoder_stride": 16,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 768,
  "id2label": {
    "0": "apple_pie",
    "1": "baby_back_ribs",
    "2": "baklava",
    "3": "beef_carpaccio",
    "4": "beef_tartare",
    "5": "beet_salad",
    "6": "beignets",
    "7": "bibimbap",
    "8": "bread_pudding",
    "9": "breakfast_burrito",
    "10": "bruschetta",
    "11": "caesar_salad",
    "12": "cannoli",
    "13": "caprese_salad",
    "14": "carrot_cake",
    "15": "ceviche",
    "16": "cheese_plate",
    "17": "cheesecake",
    "18": "chicken_curry",
    "19": "chicken_quesadilla",
    "20": "chicken_wings",
    "21": "chocolate_cake",
    "22": "chocolate_mousse",
    "23": "churros",
    "24": "clam_chowder"

Accuracy:  0.777080648443329
precision:  0.8064961508244559
f1_score:  0.7874707819662903
recall 0.777080648443329
              precision    recall  f1-score   support

           0       0.83      0.84      0.83       304
           1       0.84      0.77      0.80       287
           2       0.88      0.69      0.77       293
           3       0.65      0.59      0.62       288
           4       0.40      0.58      0.47       292
           5       0.80      0.79      0.79       291
           6       0.77      0.76      0.77       291
           7       0.85      0.78      0.81       292
           8       0.88      0.89      0.88       293
           9       0.81      0.72      0.76       278
          10       0.82      0.84      0.83       300
          11       0.88      0.74      0.80       295
          12       0.89      0.83      0.86       299
          13       0.87      0.77      0.82       286
          14       0.86      0.78      0.82       300
          15       0

All model checkpoint weights were used when initializing ViTForImageClassification.

All the weights of ViTForImageClassification were initialized from the model checkpoint at /content/Model/Models-Train-29/checkpoint-54000.
If your task is similar to the task the model of the checkpoint was trained on, you can already use ViTForImageClassification for predictions without further training.
loading configuration file /content/Model/Models-Train-29/checkpoint-54000/preprocessor_config.json
Image processor ViTImageProcessor {
  "do_normalize": true,
  "do_rescale": true,
  "do_resize": true,
  "image_mean": [
    0.5,
    0.5,
    0.5
  ],
  "image_processor_type": "ViTImageProcessor",
  "image_std": [
    0.5,
    0.5,
    0.5
  ],
  "resample": 2,
  "rescale_factor": 0.00392156862745098,
  "size": {
    "height": 224,
    "width": 224
  }
}




Evaluation at checkpoint:  54000

Accuracy:  0.7776571932442515
precision:  0.8052362057938013
f1_score:  0.7873125282939585
recall 0.7776571932442515
              precision    recall  f1-score   support

           0       0.83      0.84      0.83       304
           1       0.86      0.76      0.81       287
           2       0.87      0.71      0.78       293
           3       0.61      0.60      0.61       288
           4       0.51      0.53      0.52       292
           5       0.76      0.79      0.78       291
           6       0.82      0.76      0.79       291
           7       0.82      0.78      0.80       292
           8       0.87      0.88      0.87       293
           9       0.78      0.73      0.75       278
          10       0.85      0.83      0.84       300
          11       0.86      0.74      0.79       295
          12       0.84      0.84      0.84       299
          13       0.78      0.77      0.77       286
          14       0.88      0.78    